In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_colwidth', None)

from emogest.preprocessing import preprocess
from emogest.tokenizer import tokenize
from gensim.models import Word2Vec

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
tweets = pd.read_csv("datasets/prepared_data.csv")

In [3]:
%time tweets["preprocessed"] = tweets["tweet"].parallel_apply(preprocess)
%time tweets["tokens"] = tweets["preprocessed"].parallel_apply(tokenize)

CPU times: user 334 ms, sys: 193 ms, total: 527 ms
Wall time: 12.3 s
CPU times: user 1.13 s, sys: 373 ms, total: 1.5 s
Wall time: 21.8 s


In [4]:
sentences = tweets["tokens"].tolist()
%time model = Word2Vec(sentences, size=256, min_count=2, workers=4, iter=100)

CPU times: user 10min 37s, sys: 10.2 s, total: 10min 47s
Wall time: 5min 41s


In [5]:
model_path = "models/tweet_embedding_256.model"
model.save(model_path)